<Header> This is is where we develop a baseline factor model that allows us to take certain factors and use them in multi-OLS regression to predict our excess return signals on a security level. These signals are absolute. </Header>

In [1]:
#First, import our packages for the database connection and dataframe access
import mysql.connector
import pandas as pd
import statsmodels.api as sm
import numpy as np
import warnings
from datetime import datetime, timedelta

In [2]:
#Ignore warnings and set max row display option
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 50)


In [3]:
# Connect to MySQL database
connection = mysql.connector.connect(
    host="ubctg.con7266gcvin.us-east-2.rds.amazonaws.com",
    user="admin",
    password="ubctgquant",
    database="ubctg"
)

Here, we pull monthly returns across the stock universe over 10 years from 2011 to 2021

In [4]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2017-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT * FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}'"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
universe_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
universe_df = pd.DataFrame(universe_data, columns=columns)

#close cursor and db connection
cursor.close()

True

Generate the number of observations per security in our table. This way we can remove those that do not have a full dataset

In [5]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2017-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT COUNT(PERMNO), PERMNO FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}' GROUP BY PERMNO"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
observationTable_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
observation_df = pd.DataFrame(observationTable_data, columns=columns)

#close cursor and db connection
cursor.close()

# Print the DataFrame
observation_df

,COUNT(PERMNO),PERMNO
0,38,10026
1,38,10028
2,38,10032
3,38,10044
4,38,10051
...,...,...
9669,38,93427
9670,3,93428
9671,38,93429
9672,38,93434


Here, we remove securities that do not have enough observations (in our case, we look for at least 120)

In [6]:
#Filter securities with at least 120 observations in the period
observation_df_filtered = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >=37]

#Inner join our dataframes to only keep the securities we have data on
universe_df_filtered = pd.merge(universe_df, observation_df_filtered, on='PERMNO', how='inner')

We can now add a few factors to our model, in this case we will use GDP, CPI (inflation data), and the unemployment rate in each month. We then add it to our monthly returns dataframe for our OLS regression

In [7]:
#Introduce our factor data below as a CSV (pandas datareader not currently working for FRED api)
macro_factors = pd.read_csv("UBCTG Factor Model Example - Macro FRED Data.csv")

#These dates are at the beginning of month, so we will operate on our monthly return dataframe to convert our dates to the beginning of the month so that we can append on index
universe_df_filtered["date"] = pd.to_datetime(universe_df_filtered["date"]).dt.to_period('M').dt.to_timestamp()

#Convert macro factors date column to type datetime64 and drop the non-datetime64 column
macro_factors["date"] = pd.to_datetime(macro_factors["DATE"])
macro_factors = macro_factors.drop('DATE', axis=1)

#Inner-join macro factors dataframe with universe dataframe, using 'date' column as index
universe_df_with_external_factors = pd.merge(universe_df_filtered, macro_factors, on= 'date', how='inner')

#Ensure no errors in the returns column (there have been some instances where returns have taken on non-numeric values)
universe_df_with_external_factors["RET"] = pd.to_numeric(universe_df_with_external_factors["RET"], errors="coerce")

In [8]:
#Create a blank row at the end of our dataframe to accomodate shifted data
universe_df_with_external_factors_elongated = pd.concat([universe_df_with_external_factors, pd.DataFrame([[np.nan] * universe_df_with_external_factors.shape[1]], columns=universe_df_with_external_factors.columns)], ignore_index=True)

#Shift our returns data forward by one period. This way, we regress "t" factors to "t+1" returns, and our betas become forecasts
universe_df_with_external_factors_elongated['RET'] = universe_df_with_external_factors_elongated['RET'].shift(1)

#We want to remove our minimum date so that we have an equal number of periods across securities. Here we find the minimum date, add a day, and remove rows with that date from our dataframe
dtrmval = np.min(universe_df_with_external_factors_elongated.date)
dtrmvalrangemax = dtrmval + timedelta(days=1)

#Filter out our beginning date
universe_df_with_external_factors_filtered = universe_df_with_external_factors_elongated[(universe_df_with_external_factors_elongated["date"] >= dtrmvalrangemax)]

In [13]:
#columns_to_regress = ['Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted','CPI (USACPALTT01CTGYM)','UNRATE']

#Generate our list of unique tickers using observation df
listofPERMNO = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >=37]
uniqueTickerList = listofPERMNO['PERMNO']

In [15]:
#Initialize large df to drop results of regression for each security at each regression date
containerdf = pd.DataFrame()

#Set our lookback window to 24 periods (months in this case). This means that we will run a regression for each period (after the first 24 months) using the previous 24 months as data
LookBack_Window=24

for ticker in uniqueTickerList:
    ticker_specific_universe_df = universe_df_with_external_factors_filtered.loc[universe_df_with_external_factors_filtered['PERMNO'] == ticker]
    ticker_specific_universe_df["B0"] = 0
    ticker_specific_universe_df["R_squared"] = 0
    ticker_specific_universe_df["Beta_CPI"] = 0
    ticker_specific_universe_df["Beta_GDP"] = 0
    ticker_specific_universe_df["Beta_UnemRate"] = 0
    ticker_specific_universe_df["p_value_Alpha"] = 0
    ticker_specific_universe_df["p_value_Market"] = 0
    ticker_specific_universe_df["p_value_GDP"] = 0
    ticker_specific_universe_df["p_value_UnemRate"] = 0
    ticker_specific_universe_df= ticker_specific_universe_df.drop(["FACSHR","ACPERM","ACCOMP","DLPRC","BID","ASK","COUNT(PERMNO)","DCLRDT","DLPDT","PAYDT","RCRDDT","FACPR","FACSHR","ACPERM","ACCOMP","DLPRC"],axis=1)
    
    #For each 24-month (2-year) span, train an OLS and collect the results 
    for x in range(0, len(ticker_specific_universe_df)-LookBack_Window-1):
        # Define the independent variables (X) and dependent variable (y)
        X = ticker_specific_universe_df[["Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted","CPI (USACPALTT01CTGYM)","UNRATE"]][x:x+LookBack_Window]
        Y = ticker_specific_universe_df['RET'][x:x+LookBack_Window]

        #Add a constant term to the independent variables, check impact
        X = sm.add_constant(X)
    
        #Fit the linear regression model
        model = sm.OLS(Y, X)
        results = model.fit()
        
        #Get the coefficients
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'B0'] = results.params[0]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_GDP'] = results.params[1]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_CPI'] = results.params[2]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'Beta_UnemRate'] = results.params[3]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'R_squared'] = results.rsquared
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_B0'] = results.pvalues[0]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_GDP'] = results.pvalues[1]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_CPI'] = results.pvalues[2]
        ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], 'p_value_UnemRate'] = results.pvalues[3]
        
    #Add ticker dataframe to larger container dataframe
    containerdf = pd.concat([containerdf, ticker_specific_universe_df], ignore_index=True)
display(containerdf)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


,PERMNO,date,COMNAM,NAICS,PERMCO,PRC,VOL,RET,UNRATE,CPI (USACPALTT01CTGYM),...,R_squared,Beta_CPI,Beta_GDP,Beta_UnemRate,p_value_Alpha,p_value_Market,p_value_GDP,p_value_UnemRate,p_value_B0,p_value_CPI
0,10026.0,2018-01-01,J & J SNACK FOODS CORP,311821,7976.0,138.44000,17680.0,0.007743,4.0,2.042188,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
1,10026.0,2018-02-01,J & J SNACK FOODS CORP,311821,7976.0,134.33000,16663.0,-0.088191,4.1,2.175117,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
2,10026.0,2018-03-01,J & J SNACK FOODS CORP,311821,7976.0,136.56000,14156.0,-0.029688,4.0,2.321362,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
3,10026.0,2018-04-01,J & J SNACK FOODS CORP,311821,7976.0,137.41000,12510.0,0.019951,4.0,2.406737,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
4,10026.0,2018-05-01,J & J SNACK FOODS CORP,311821,7976.0,141.62000,15898.0,0.006224,3.8,2.725657,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230452,93436.0,2020-09-01,TESLA INC,336111,53453.0,429.01001,17331954.0,0.741452,7.8,1.332004,...,0.073800,0.063205,0.002246,0.022909,0,0,0.314632,0.474311,0.709734,0.687477
230453,93436.0,2020-10-01,TESLA INC,336111,53453.0,388.04001,8330610.0,-0.139087,6.8,1.130982,...,0.148418,0.173466,0.002719,0.043094,0,0,0.184883,0.201254,0.343483,0.316921
230454,93436.0,2020-11-01,TESLA INC,336111,53453.0,567.59998,7811501.0,-0.095499,6.7,1.122155,...,0.172758,0.215318,0.003272,0.055180,0,0,0.167997,0.162484,0.316074,0.294730
230455,93436.0,2020-12-01,TESLA INC,336111,53453.0,705.66998,11962716.0,0.462736,6.7,1.323012,...,0.250814,0.394869,0.003239,0.083113,0,0,0.153381,0.041649,0.080240,0.068731


In [16]:
display(containerdf)

,PERMNO,date,COMNAM,NAICS,PERMCO,PRC,VOL,RET,UNRATE,CPI (USACPALTT01CTGYM),...,R_squared,Beta_CPI,Beta_GDP,Beta_UnemRate,p_value_Alpha,p_value_Market,p_value_GDP,p_value_UnemRate,p_value_B0,p_value_CPI
0,10026.0,2018-01-01,J & J SNACK FOODS CORP,311821,7976.0,138.44000,17680.0,0.007743,4.0,2.042188,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
1,10026.0,2018-02-01,J & J SNACK FOODS CORP,311821,7976.0,134.33000,16663.0,-0.088191,4.1,2.175117,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
2,10026.0,2018-03-01,J & J SNACK FOODS CORP,311821,7976.0,136.56000,14156.0,-0.029688,4.0,2.321362,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
3,10026.0,2018-04-01,J & J SNACK FOODS CORP,311821,7976.0,137.41000,12510.0,0.019951,4.0,2.406737,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
4,10026.0,2018-05-01,J & J SNACK FOODS CORP,311821,7976.0,141.62000,15898.0,0.006224,3.8,2.725657,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230452,93436.0,2020-09-01,TESLA INC,336111,53453.0,429.01001,17331954.0,0.741452,7.8,1.332004,...,0.073800,0.063205,0.002246,0.022909,0,0,0.314632,0.474311,0.709734,0.687477
230453,93436.0,2020-10-01,TESLA INC,336111,53453.0,388.04001,8330610.0,-0.139087,6.8,1.130982,...,0.148418,0.173466,0.002719,0.043094,0,0,0.184883,0.201254,0.343483,0.316921
230454,93436.0,2020-11-01,TESLA INC,336111,53453.0,567.59998,7811501.0,-0.095499,6.7,1.122155,...,0.172758,0.215318,0.003272,0.055180,0,0,0.167997,0.162484,0.316074,0.294730
230455,93436.0,2020-12-01,TESLA INC,336111,53453.0,705.66998,11962716.0,0.462736,6.7,1.323012,...,0.250814,0.394869,0.003239,0.083113,0,0,0.153381,0.041649,0.080240,0.068731
